# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
from selenium import webdriver
driver = webdriver.Chrome()

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [2]:
driver.get('https://www.tdlr.texas.gov/cimsfo/fosearch.asp')

In [3]:
# find field for last name
lname_input = driver.find_element_by_name('pht_lnm')

In [4]:
lname_input.send_keys('Nguyen')

In [5]:
search_button = driver.find_element_by_name('B1')
search_button.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [6]:
results = driver.find_elements_by_tag_name('tr')

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

In [12]:
for result in results[:10]:
    try:
        name = result.find_element_by_class_name('results_text')
        print(name.text)
    except:
        print("No name.")

No name.
NGUYEN, TOAN HUU
NGUYEN, HANH CONG
NGUYEN, KHIEM VAN
NGUYEN, DIEP THI NGOC
NGUYEN, LAN T-THUY
NGUYEN, TUAN A
NGUYEN, THAO B
NGUYEN, BETH MARIA
NGUYEN, KENNEY TUAN


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...

In [56]:
for result in results[:10]:
    try:
        violation = result.find_elements_by_tag_name('td')
        print(violation[2].text)
    except:
        print("No violation.")

# also works by changing for loop to:
# for result in results[1:11]

No violation.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect all wax pots.
Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution; Respondent failed to disinfect multi-use equipment, implements, and tools prior to use on each client.
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
Respondent failed to clean and disinfect all wax pots; Respondent failed to disinfect tools, 

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [63]:
for result in results[:10]:
    try:
        complaint_id = result.find_elements_by_class_name('results_text')
        print(complaint_id[-2].text)
    except:
        print("No complaints.")

No complaints.
COS20180004289
COS20180006594
COS20180000257
COS20180004915
COS20180009255
COS20140018343
COS20180008846
COS20180000897
BAR20180001231


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [74]:
rows = []

# for loop to pull out each category of info from the first 10 search results
for result in results[1:11]:
    tx_cos = {}
    
    # name
    try:
        tx_cos['name'] = result.find_element_by_class_name('results_text').text        
    except:
        tx_cos['name'] = "No name."
    
    # violation
    try:
        violation_desc = result.find_elements_by_tag_name('td')
        tx_cos['violation_desc'] = violation_desc[2].text
    except:
        print("No violation.")
    
    # violation/ complaint number
    try:
        complaint_id = result.find_elements_by_class_name('results_text')
        tx_cos['violation_number'] = complaint_id[-2].text
    except:
        print("No complaints.")
    
    # license numbers
    try:
        license_number = result.find_elements_by_class_name('results_text')
        tx_cos['license_number'] = license_number[4].text
    except:
        print("No licenses.")
        
    # zip code
    try:
        zip_code = result.find_elements_by_class_name('results_text')
        tx_cos['zip_code'] = zip_code[3].text
    except:
        print("No zip code.")
        
    # county
    try:
        county = result.find_elements_by_class_name('results_text')
        tx_cos['county'] = county[2].text
    except:
        print("No county.")

    # city
    try:
        city = result.find_elements_by_class_name('results_text')
        tx_cos['city'] = city[1].text
    except:
        print("No city.")

    rows.append(tx_cos)
rows

[{'name': 'NGUYEN, TOAN HUU',
  'violation_desc': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.',
  'violation_number': 'COS20180004289',
  'license_number': '780948, 1706491, 1699123',
  'zip_code': '78217',
  'county': 'BEXAR',
  'city': 'SAN ANTONIO'},
 {'name': 'NGUYEN, HANH CONG',
  'violation_desc': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.',
  'violation_number': 'COS20180006594',
  'license_number': '737708',
  'zip_code': '79934',
  'county': 'EL PASO',
  'city': 'EL PASO'},
 {'name': 'NGUYEN, KHIEM VAN',
  'violation_desc': 'Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to 

### Save that to a CSV

- Tip: You'll want to use pandas here

In [75]:
import pandas as pd
df = pd.DataFrame(rows)
df

,city,county,license_number,name,violation_desc,violation_number,zip_code
0,SAN ANTONIO,BEXAR,"780948, 1706491, 1699123","NGUYEN, TOAN HUU",Respondent failed to clean and sanitize whirlp...,COS20180004289,78217
1,EL PASO,EL PASO,737708,"NGUYEN, HANH CONG",Respondent failed to clean and sanitize whirlp...,COS20180006594,79934
2,LONGVIEW,GREGG,731665,"NGUYEN, KHIEM VAN",Respondent failed to follow whirlpool foot spa...,COS20180000257,75604
3,HOUSTON,HARRIS,"1347649, 760528","NGUYEN, DIEP THI NGOC","Respondent failed to disinfect tools, implemen...",COS20180004915,77014
4,SAN ANTONIO,BEXAR,"NGUYEN, SAMLOI","NGUYEN, LAN T-THUY","Respondent failed to clean, disinfect, and ste...",COS20180009255,78255
5,AUSTIN,TRAVIS,"NGUYEN, TUAN VAN","NGUYEN, TUAN A",Respondent failed to clean and disinfect all w...,COS20140018343,78723
6,EULESS,TARRANT,"721373, 1142884","NGUYEN, THAO B",Respondent failed to clean and sanitize whirlp...,COS20180008846,76039
7,HOUSTON,HARRIS,1470271,"NGUYEN, BETH MARIA",The Respondent's license was revoked upon Resp...,COS20180000897,77083
8,CEDAR PARK,WILLIAMSON,692892,"NGUYEN, KENNEY TUAN",Respondent leased space in a barber shop to an...,BAR20180001231,78613
9,AMARILLO,POTTER,"1196244, 767015, 767014","NGUYEN, TRUNG N","Respondent failed to clean, disinfect, and ste...",COS20170023893,79106


In [77]:
df.to_csv("texas-cosmetology-violations.csv", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.